### We start by importing our libraries 


In [1]:
!pip install BeautifulSoup4
!pip install requests


     |████████████████████████████████| 102kB 17.0MB/s ta 0:00:01


In [2]:
import numpy as np 
import pandas as pd 
from pandas.io.json import json_normalize
import json
import requests
from bs4 import BeautifulSoup 
from sklearn.cluster import KMeans 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium


We then import the data (this data is from week 3) , join it and changed it to a data frame

In [27]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

text_result = requests.get(url).text 
html_parsed_result = BeautifulSoup(text_result, 'html.parser')

Neighborhood_info_table = html_parsed_result.find('table', class_ = 'wikitable')
Neighborhood_rows = Neighborhood_info_table.find_all('tr')

Neighborhood_info = []
for row in Neighborhood_rows:
    info = row.text.split('\n')[1:-1] 
    Neighborhood_info.append(info)
Neighborhood_df = pd.DataFrame(Neighborhood_info[1:], columns=Neighborhood_info[0])
not_assigned_boroughs = Neighborhood_df.index[Neighborhood_df['Borough'] == 'Not assigned']
not_assigned_neighbourhoods = Neighborhood_df.index[Neighborhood_df['Neighbourhood'] == 'Not assigned']
not_assigned_neighbourhoods_and_boroughs = not_assigned_boroughs & not_assigned_neighbourhoods
Neighborhood_df.drop(Neighborhood_df.index[not_assigned_boroughs], inplace=True)
Neighborhood_df.reset_index(drop=True, inplace=True)
not_assigned_neighbourhoods = Neighborhood_df.index[Neighborhood_df['Neighbourhood'] == 'Not assigned'] 
for idx in not_assigned_neighbourhoods:
    Neighborhood_df['Neighbourhood'][idx] = Neighborhood_df['Borough'][idx]
Neighborhood_df = Neighborhood_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
Neighborhood_df = Neighborhood_df.sample(frac=1).reset_index(drop=True)

In [35]:
url2="http://cocl.us/Geospatial_data"
geo_data=pd.read_csv(url2)
geo_data.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_toronto=pd.merge(Neighborhood_df,geo_data,on='Postcode')
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
1,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
2,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
3,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
4,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302


We now map the city of Toronto with all its postal codes regions using the latitude and longtitude given by google and our imported folium map

In [38]:
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

We focus now on the borough of Scarborough 

In [39]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True)
scarborough_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
1,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302
2,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
3,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
4,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Let's create a map of Scarborough and all it's neighborhoods

In [40]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

We now use foursqaure data base to obtain data on the venues of each neighborhood

In [41]:
# @hidden_cell
CLIENT_ID = 'MITOL0ANQWPEHHS4FAEWNASU1EG4N2LFWVFCMTWQNYS0211P' 
CLIENT_SECRET = 'H3PBT4F1ZISCILN3AX0CFQ1WMEN043PJIRPPQXDA3AARBRKG' 
VERSION = '20181102'

In [43]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;
print('Crawling different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Postcode']),
                                                   list(scarborough_data['Neighbourhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Crawling different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1T (and Neighborhoods Clarks Corners, Sullivan, Tam O'Shanter) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffcrest, Cliffside, Scarborough Village West) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1N (and Neighborhoods Birch Cliff, Cliffside West) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Highland Creek, Rouge Hill, Port Union) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1S (and Neighborhoods Agincourt) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1L (and Neighborhoods Clairlea, Golden Mile, Oakridge) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1V (and Neighborhoods Agincourt North, L'Amor

We clean the data received from Foursquare Database

In [45]:

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
     
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
    return(result_df)

In [46]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
31
Number of Venuse in Coordination "M1T" Posal Code and "Clarks Corners, Sullivan, Tam O'Shanter" Negihborhood(s) is:
35
Number of Venuse in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is:
12
Number of Venuse in Coordination "M1M" Posal Code and "Cliffcrest, Cliffside, Scarborough Village West" Negihborhood(s) is:
13
Number of Venuse in Coordination "M1N" Posal Code and "Birch Cliff, Cliffside West" Negihborhood(s) is:
16
Number of Venuse in Coordination "M1C" Posal Code and "Highland Creek, Rouge Hill, Port Union" Negihborhood(s) is:
5
Number of Venuse in Coordination "M1S" Posal Code and "Agincourt" Negihborhood(s) is:
46
Number of Venuse in Coordination "M1L" Posal Code and "Clairlea, Golden Mile, Oakridge" Negihborhood(s) is:
29
Number of Venuse in Coordination "M1V" Posal Code and "Agincourt North, L'Amoreaux East, Milliken, Steeles East" Negihborhood(s) is:
25
Number

We explore all of the venues in each neighborhood

In [47]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1H,Cedarbrae,43.773136,-79.239476,Federick Restaurant,This spot is popular,Hakka Restaurant,219
1,M1H,Cedarbrae,43.773136,-79.239476,CANBE Foods Inc,This spot is popular,Indian Restaurant,532
2,M1H,Cedarbrae,43.773136,-79.239476,Drupati's Roti & Doubles,This spot is popular,Caribbean Restaurant,291
3,M1H,Cedarbrae,43.773136,-79.239476,The Real McCoy Burgers And Pizza,This spot is popular,Burger Joint,729
4,M1H,Cedarbrae,43.773136,-79.239476,Tim Hortons,This spot is popular,Coffee Shop,670


In [48]:
scarborough_venues.to_csv('scarborough_venues.csv')
scarborough_venues = pd.read_csv('scarborough_venues.csv')
neighborhood_list = list(scarborough_venues['Neighborhood'].unique())
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()



,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,46,46,46,46,46,46,46
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",25,25,25,25,25,25,25
"Birch Cliff, Cliffside West",16,16,16,16,16,16,16
Cedarbrae,31,31,31,31,31,31,31
"Clairlea, Golden Mile, Oakridge",29,29,29,29,29,29,29


In [49]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 117 uniques categories.
Here is the list of different categories:


['Hakka Restaurant',
 'Indian Restaurant',
 'Caribbean Restaurant',
 'Burger Joint',
 'Coffee Shop',
 'Thai Restaurant',
 'Chinese Restaurant',
 'Music Store',
 'Athletics & Sports',
 'Pharmacy',
 'Bank',
 'Bakery',
 'Fried Chicken Joint',
 'Yoga Studio',
 'Wings Joint',
 'Grocery Store',
 'Fast Food Restaurant',
 'Flower Shop',
 'Asian Restaurant',
 'Rental Car Location',
 'Bus Line',
 'Sporting Goods Shop',
 'Martial Arts Dojo',
 'Pizza Place',
 'Pet Store',
 'Italian Restaurant',
 'Noodle House',
 'Mexican Restaurant',
 'Seafood Restaurant',
 'Greek Restaurant',
 'Cantonese Restaurant',
 'Sandwich Place',
 'Vietnamese Restaurant',
 'Discount Store',
 'Intersection',
 'Park',
 'Shopping Mall',
 'Golf Course',
 'Hobby Shop',
 'Taiwanese Restaurant',
 'Deli / Bodega',
 'Video Game Store',
 'Market',
 'Restaurant',
 'Convenience Store',
 'Auto Garage',
 'Train Station',
 'Japanese Restaurant',
 'Bowling Alley',
 'Beach',
 'Furniture / Home Store',
 'Sports Bar',
 'Cajun / Creole Restaur

In [50]:
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,African Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hobby Shop,Hotpot Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,0,M1H,Cedarbrae,43.773136,-79.239476,Federick Restaurant,This spot is popular,219,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1H,Cedarbrae,43.773136,-79.239476,CANBE Foods Inc,This spot is popular,532,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1H,Cedarbrae,43.773136,-79.239476,Drupati's Roti & Doubles,This spot is popular,291,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1H,Cedarbrae,43.773136,-79.239476,The Real McCoy Burgers And Pizza,This spot is popular,729,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1H,Cedarbrae,43.773136,-79.239476,Tim Hortons,This spot is popular,670,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'African Restaurant',
 'American Restaurant',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Diner',
 'Fast Food Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Fruit & Vegetable Store',
 'Greek Restaurant',
 'Grocery Store',
 'Hakka Restaurant',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Noodle House',
 'Pizza Place',
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shanghai Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
 'Wings Joint']

In [55]:
scarborough_onehot = scarborough_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()

scarborough_onehot.head()

,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Greek Restaurant,Grocery Store,Hakka Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,0,0,0,2,1,0,0,1,2,8,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,2,1,1,1,1,1,0,0,0,1,0
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",0,0,0,1,1,0,0,0,0,1,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,2,2,0,0,0,0,0,0,0,1,0,0
"Birch Cliff, Cliffside West",0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
Cedarbrae,0,0,1,0,3,0,1,0,0,1,1,0,1,0,0,1,0,0,1,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
"Clairlea, Golden Mile, Oakridge",0,0,0,0,2,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0


We now integrate all different joints and restaurant

In [56]:
feat_name_list = list(scarborough_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
scarborough_onehot['Total Restaurants'] = scarborough_onehot[restaurant_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = restaurant_list)


feat_name_list = list(scarborough_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)
scarborough_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,
Agincourt,2,1,0,0,0,0,1,1,2,1,19,0
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,0,0,0,0,0,0,2,2,0,9,1
"Birch Cliff, Cliffside West",0,0,1,0,0,0,0,0,0,0,4,0
Cedarbrae,3,0,0,0,0,0,1,0,1,0,8,3
"Clairlea, Golden Mile, Oakridge",2,0,0,0,0,0,1,0,1,1,4,0
"Clarks Corners, Sullivan, Tam O'Shanter",0,0,0,0,0,0,1,1,2,2,12,1
"Cliffcrest, Cliffside, Scarborough Village West",0,0,0,0,0,0,0,0,3,0,3,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",1,0,0,0,0,0,1,0,1,1,12,3
"East Birchmount Park, Ionview, Kennedy Park",0,0,0,0,0,0,2,0,2,1,6,1


We now run k-means to cluster the neighborhoods using k=5

In [57]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

In [58]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G1,2.0,1.000000,0.000000,0.0,0.0,0.000000,1.000000,1.0,2.000000,1.0,19.000000,0.000000,27.0
G5,0.5,0.000000,0.000000,0.0,0.0,0.000000,1.000000,0.5,1.500000,1.5,12.000000,2.000000,19.0
G2,1.4,0.400000,0.000000,0.2,0.0,0.000000,1.200000,0.4,1.600000,0.4,7.600000,1.400000,14.6
G4,0.0,0.000000,0.000000,0.0,0.5,0.000000,0.000000,0.0,4.000000,0.5,3.000000,2.000000,10.0
G3,0.5,0.166667,0.166667,0.0,0.0,0.166667,0.166667,0.0,0.333333,0.5,3.833333,0.166667,6.0


So G1 one is the best group

We then find the coressponding group to each neighborhood

In [62]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,1
1,"Agincourt North, L'Amoreaux East, Milliken, St...",2
2,"Birch Cliff, Cliffside West",3
3,Cedarbrae,2
4,"Clairlea, Golden Mile, Oakridge",3
5,"Clarks Corners, Sullivan, Tam O'Shanter",5
6,"Cliffcrest, Cliffside, Scarborough Village West",4
7,"Dorset Park, Scarborough Town Centre, Wexford ...",5
8,"East Birchmount Park, Ionview, Kennedy Park",2
9,"Guildwood, Morningside, West Hill",4


So the best neighborhoods are :

In [64]:
neigh_summary[neigh_summary['Group'] == 1]

,Neighborhood,Group
0,Agincourt,1


In [65]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 1]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1S',
 'Neighborhood': 'Agincourt',
 'Neighborhood Latitude': 43.7942003,
 'Neighborhood Longitude': -79.26202940000002}

Second Best :

In [66]:
neigh_summary[neigh_summary['Group'] == 5]

,Neighborhood,Group
5,"Clarks Corners, Sullivan, Tam O'Shanter",5
7,"Dorset Park, Scarborough Town Centre, Wexford ...",5


Third best:

In [67]:
neigh_summary[neigh_summary['Group'] == 2]

,Neighborhood,Group
1,"Agincourt North, L'Amoreaux East, Milliken, St...",2
3,Cedarbrae,2
8,"East Birchmount Park, Ionview, Kennedy Park",2
11,L'Amoreaux West,2
12,"Maryvale, Wexford",2


### The End
### Thanks for reading